## AIF_sprint12-cnn
【この課題の目的】      
スクラッチを通してCNNの基礎を理解する     
【目的としないこと】    
実用的なCNNモデルに関する知識の習得

## 小さな配列での1次元畳み込み層の実験

In [1]:
import pandas as pd
import numpy as np

X = np.array([1,2,3,4])

In [44]:
# チャンネル数を1に限定した1次元畳み込み層のクラス
class SimpleConv1d:
    """
    ノード数n_nodes1からn_nodes2への全結合層
    Parameters
    ----------
    n_nodes1 : int
      前の層のノード数
    n_nodes2 : int
      後の層のノード数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    """
    # FCクラスを作り替える
    
    def __init__(self):
        # 引数としてinitializerクラスのインスタンス変数を受け取る
        # フィルタとバイアスの初期値
        
        self.W = np.array([3,5,7])
        self.B = np.array([1])
        self.pad = 0
        self.stride = 1
        self.delta_a = np.array([10,20])
        
        
    def forward(self, X):
        a_list = []
        count = 0
        #1*3のフィルタを定義
        for i in range(len(X)-1):
            # 出力のセル1個分
            r = 0
            for s in range(self.W.shape[0]):
                if i +s >= X.shape[0]:
                    return np.array(a_list)
                r += np.dot(X[i+s],self.W[s])
            # バイアス項を加算
            r += self.B
            a_list.append(r)
        
    def backward(self, X):
        delta_b = np.sum(self.delta_a,axis=0,keepdims=True)
        delta_w = []
        delta_x = []
        # Wの勾配を計算
        for i in range(self.W.shape[0]):
            index = np.arange(i,i+2)
            delta_w.append(np.dot(self.delta_a,X[index]))
            
        # Xの勾配を計算
        for i in range(X.shape[0]):
        # インデックスがXの始めと末尾のみ、別の処理
            if i ==0:
                delta_x.append(self.delta_a[i]*self.W[i])
            elif i ==len(X)-1:
                delta_x.append(self.delta_a[i-2]*self.W[i-1])
            else:
                index = np.array([i,i-1])
                delta_x.append(np.dot(self.delta_a,self.W[index]))
        delta_w = np.array(delta_w)
        delta_x = np.array(delta_x)
        
        return delta_b,delta_w,delta_x
    

In [45]:
model = SimpleConv1d()

In [46]:
model.forward(X)

array([[35],
       [50]])

In [47]:
model.backward(X)

(array([30]), array([ 50,  80, 110]), array([ 30, 110, 170, 140]))

In [6]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.datasets import fetch_mldata

# mnist_dir = "/Users/andoutakaaki/DIC_study/sprint9/mnist_data/"

# # MNISTの読み込み
# mnist = fetch_mldata('MNIST original', data_home=mnist_dir)

# # trainとtestに分割
# # MNISTの読み込み
# X_train, X_test, y_train, y_test = train_test_split(mnist.data, mnist.target, test_size=0.2)
# # さらにtrainとvalに分割
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# # ラベルをint型にしておく
# y_train = y_train.astype(np.int)
# y_test = y_test.astype(np.int)
# y_val = y_val.astype(np.int)

# X_train = X_train.astype(np.float)
# X_test = X_test.astype(np.float)
# X_val = X_val.astype(np.float)
# X_train /= 255
# X_test /= 255
# X_val /= 255

# from sklearn.preprocessing import OneHotEncoder
# enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
# y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
# y_test_one_hot = enc.transform(y_test[:, np.newaxis])
# y_val_one_hot = enc.transform(y_val[:, np.newaxis])

# print(X_train.shape,X_test.shape,X_val.shape)
# print(y_train_one_hot.shape,y_test_one_hot.shape,y_val_one_hot.shape)


In [7]:
# --各設定値--
img_h = 28
img_w = 28
img_ch = 1

wb_width = 0.1
alpha = 0.01
num_epochs = 10
batch_size = 10
inteval = 10
n_samples = 44800

In [8]:
# def __forward(self, X):
#         s_batch, k, xh, xw = X.shape
#         m = self.weight.shape[0]
#         oh, ow = xh - self.kh / 2 * 2, xw - self.kw / 2 * 2
#         self.__patch = self.__im2patch(X, s_batch, k, oh, ow)
#         return np.tensordot(self.__patch, self.weight, ((2, 3, 4), (1, 2, 3))).swapaxes(1, 2).reshape(s_batch, m, oh, ow)


# def __im2patch(self, X, s_batch, k, oh, ow):
#         patch = np.zeros((s_batch, oh * ow, k, self.kh, self.kw))
#         for j in range(oh):
#             for i in range(ow):
#                 patch[:, j * ow + i, :, :, :] = X[:, :, j:j+self.kh, i:i+self.kw]
#         return patch
    
# def backward(self, delta, shape):
#         s_batch, k, h, w = delta.shape
#         delta_patch = np.tensordot(delta.reshape(s_batch, k, h * w), self.weight, (1, 0))
#         return self.__patch2im(delta_patch, h, w, shape)

# def __patch2im(self, patch, h, w, shape):
#         im = np.zeros(shape)
#         for j in range(h):
#             for i in range(w):
#                 im[:, :, j:j+self.kh, i:i+self.kw] += patch[:, j * w + i]
#         return im
    


In [9]:
# #  im2colのアルゴリズム
# def im2col(x, w.shape[0], w.shape[1], y.shape[0], y.shape[1]):
#     x.shape[0], x.shape[1] = x.shape
#     cols = np.zeros((w.shape[0], w.shape[1], y.shape[0], y.shape[1]))
    
#     for h in range(w.shape[0]):
#         h_lim = h + y.shape[0]
#         for w in range(w.shape[1]):
#             w_lim = w + y.shape[1]
#             cols[h, w, :, :] = img[h:h_lim, w:w_lim]
            
#         cols = cols.reshape(w.shape[0]*w.shape[1], y.shape[0]*y.shape[1])
        
#         return cols

In [10]:
# #  col2imのアルゴリズム
# def col2im(cols, img_shape, flt_h, flt_w, out_h, out_w):
    
#     img_h, img_w = img_shape
#     cols = cols.reshape(flt_h, flt_w, out_h, out_w, ).transpose
    
    

In [11]:
# #  im2colのアルゴリズム
# def im2col(image, flt_h, flt_w, out_h, out_w):
    
#     img_h, img_w = image.shape
#     cols = np.zeros((flt_h, flt_w, out_h, out_w))
    
#     for h in range(flt_h):
#         h_lim = h + out_h
#         for w in range(flt_w):
#             w_lim = w + out_w
#             cols[h, w, :, :] = img[h:h_lim, w:w_lim]
            
#         cols = cols.reshape(flt_h*flt_w, out_h*out_w)
        
#         return cols
    